# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [1]:
import matplotlib.pyplot as plt
import PIL.Image as PIL_image
import numpy as np
import cv2
from scipy.io import loadmat,savemat

from importlib.machinery import SourceFileLoader
#In case of importing files from other directory
cotrakcer_pb2 = SourceFileLoader("cotracker_pb2","../protos/cotracker_pb2.py").load_module()
cotrakcer_pb2_grpc = SourceFileLoader("cotracker_pb2_grpc","../protos/cotracker_pb2_grpc.py").load_module()

import argparse
import grpc
import io

## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data (image in a matfile) and call the service

In [24]:
from PIL import Image
import decord
import torch

#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'

# List of file paths (example paths — replace with your actual ones)
video_path = "./cut.mp4"
send_video = False

#send video as file bytes
if send_video:
    with open(video_path, "rb") as f:
        video_bytes = f.read()

    request = cotrakcer_pb2.CoTrackerRequest(
        video=video_bytes,
        grid_size=32)
else:
    #send video as tensor
    
    video = decord.VideoReader(video_path)
    frames = [video[i].asnumpy() for i in range(len(video))]
    video_numpy = np.stack(frames) # shape (num_frames, height, width, 3)

    #goto 1, frames, 3, height, width
    video_numpy = np.transpose(video_numpy,(0,3,1,2)) # shape (num_frames, 3, height, width)
    video_tensor = torch.tensor(video_numpy).unsqueeze(0).float()

channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = cotrakcer_pb2_grpc.CoTrackerServiceStub(channel)

#### Do your task. Example: read an image from a matlab file and send it to process

In [25]:
#Name of the method in the service
response = estimator_stub.Forward(request)
#After finishing channel.close

In [29]:
import torch
torch.load(io.BytesIO(response.tracks))
#torch.load(io.BytesIO(response.visibility))

tensor([[[[  20.2740,   15.0183],
          [  60.8219,   15.0183],
          [ 101.3699,   15.0183],
          ...,
          [1196.1644,  705.8590],
          [1236.7124,  705.8590],
          [1277.2604,  705.8590]],

         [[  20.9392,   15.1378],
          [  61.1122,   15.7791],
          [ 100.8960,   16.0372],
          ...,
          [1199.1458,  706.3630],
          [1239.1407,  706.5099],
          [1278.1276,  706.2202]],

         [[  20.2246,   14.9090],
          [  59.9206,   15.5971],
          [  99.1393,   15.8833],
          ...,
          [1199.5540,  705.9616],
          [1239.5835,  706.1919],
          [1278.2189,  705.6882]],

         ...,

         [[  18.3808,   15.4969],
          [  56.3232,   16.6860],
          [  88.7253,   17.6858],
          ...,
          [1251.0031,  654.7152],
          [1291.4117,  663.1351],
          [1320.6698,  651.7093]],

         [[  19.7382,   16.3766],
          [  56.7336,   18.5109],
          [  87.8291,   20.4812],

I0000 00:00:1757416848.279901   57566 chttp2_transport.cc:1335] ipv6:%5B::1%5D:8061: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {grpc_status:14, http2_error:2}
